# 0. Setup

In [1]:
import pandas as pd
import os.path
import math
import re
import random

In [2]:
root_path = os.path.dirname(os.getcwd())
ingredients_data_raw = pd.read_csv(os.path.join(root_path, 'DATA/ingredients_data.csv'))
ingredients_data = ingredients_data_raw.replace(float('nan'), '')
stir_fry_data_impractical = ingredients_data[ingredients_data['stir_fry'] =='y']
stir_fry_data = ingredients_data[(ingredients_data['stir_fry_yes'] == 'y') & (ingredients_data['salad_umbrella'] != 'y') & (ingredients_data['redirect'] != 'y')]

stir_fry_data_basic = stir_fry_data[stir_fry_data['stir_fry_basic'] == 'y']
# stir_fry_data = stir_fry_data_basic
stir_fry_data_is_basic = False

stir_fry_data.reset_index(inplace=True)

# 1. Matching "pairs with" terms to ingredient names

In [3]:
def get_terms_from_pairs_with(pairs_with):
    if str(pairs_with) == 'nan':
        return []
    else:
        return [term.strip() for term in pairs_with.split('\n\n') if term.strip() != '']

# break entries in column that has 'pairs with' strings into lists of ingredient terms
ingredient_pairs_with_terms = stir_fry_data['pairs_with'].apply(get_terms_from_pairs_with)

In [4]:
# create list of all terms, ignoring case and excluding duplicates
all_terms = list(set(ingredient_pairs_with_terms.sum()))
all_terms_lower = list(set([term.lower() for term in ingredient_pairs_with_terms.sum()]))

In [5]:
# !pip install inflect
import inflect
p = inflect.engine()

In [6]:
# TODO
    # replace numbers w c, C
    # auto caps
    # import salad, overwrite
    # guess at direct match
    
# how to tell if direct match:
    # if term has e.g.
        # if first name token is followed by comma then another term
            # first tokens match and term contains second name token OR term contains token, comma, token
        # else:
            # return: term contains first name token
    # else:
        # if first name token is followed by comma then another term
            # return: term contains reversed name tokens OR term contains token, comma, token
        # else:
            # return: term contains first name token
            
# how to tell if category match:
    # if term has e.g.
        # if first name token is followed by comma then another term
            # if first tokens match
                # if term contains second name token
                    # return d
                # else:
                    # return c
            # else:
                # return ''
        # else:
            # if first tokens match (before e.g. in term):
                # return c
            # elif name token in matches after e.g.
                # return d
            # else:
                # return ''
            # return: term contains first name token
    # else:
        # if first name token is followed by comma then another term
            # if term contains reversed name tokens OR term contains token, comma, token
                # return d
            # elif term contains first name token:
                # return c
            # else:
                # return ''
        # else:
            # if first name token in term:
                # return direct
            # elif any common tokens:
                # return c
            # else:
                # return ''

salad_matches = pd.read_csv(os.path.join(root_path, 'DATA/term_name_matches_specific.csv'))
    
def get_tokens(phrase):
    tokens = [token.strip() for token in re.split('\(|\)|,|e\.g\.|esp\.|and|—|or|aka|see|see also|;|and\/or|\*', phrase)]
#     print(tokens)
    tokens = [p.singular_noun(token) or token for token in tokens if token != '']
#     print(tokens)
    return tokens

def get_mark(name, term):
#     print()
#     print('NAME', name)
#     print('TERM', term)
    try:
        salad_match = salad_matches[term][salad_matches['name'] == name].iloc[0]
#         print('SALAD MATCH', salad_match)
    except:
#         print('NO SALAD MATCH')
        salad_match = None 
    if salad_match:
        if str(salad_match) in ['0', 'nan']:
#             print('BAD SALAD MATCH')
            return ''
        else:
#             print('GOOD SALAD MATCH', salad_match)
#             print('NAME', name)
#             print('TERM', term)
#             print()
            return salad_match
    
    name_tokens = [token.lower() for token in get_tokens(name)]  
    term_tokens_mixed = get_tokens(term)
    term_tokens = [token.lower() for token in term_tokens_mixed]
    
    primary_name = re.split('\(|—|e\.g\.', name)[0].strip()
    primary_name_split = primary_name.split(', ')
    single_comma_primary_name = len(primary_name_split) == 2
    
#     print()
    if 'e.g.' in term:
        if single_comma_primary_name:
            if name_tokens[0] == term_tokens[0]:
                if name_tokens[1] in term_tokens: # specific name in e.g. term
#                     print('NAME TOKENS In TERM TOKENS')
                    term_i = term_tokens.index(name_tokens[1])
                    if term_tokens_mixed[term_i] == term_tokens_mixed[term_i].upper():
                        match = 'D'
                    else:
                        if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
                            match = 'D'
                        else:
                            match = 'd'
                else: # name matches only generic (pre e.g.) part of term
                    if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
                        match = 'C'
                    else:
                        match = 'c'
            else:
                match = ''
        else:
            if name_tokens[0] == term_tokens[0]: # name matches term before e.g.
                if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
                    match = 'C'
                else:
                    match = 'c'
            elif name_tokens[0] in term_tokens[1:]: # name matches term after e.g.
                term_i = term_tokens.index(name_tokens[0])
                if term_tokens_mixed[term_i] == term_tokens_mixed[term_i].upper():
                    match = 'D'
                else:
                    if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
                        match = 'D'
                    else:
                        match = 'd'
            else:
                match = ''
    else:
        if single_comma_primary_name:
            if ' '.join(primary_name_split).lower() in term.lower() or primary_name.lower() in term.lower(): # if primary_name is in term, any order
                if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
                    match = 'D'
                else:
                    match = 'd'
            elif name_tokens[0] == term_tokens[0]: # if first part of primary name matches first token in term
                if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
                    match = 'C'
                else:
                    match = 'c'
            else:
                match = ''
        else:
            if name_tokens[0] in term_tokens: # if non-comma name anywhere in non-e.g. term_tokens
                if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
                    match = 'D'
                else:
                    match = 'd'
            elif len(set(name_tokens).intersection(set(term_tokens))) > 0: # if there are any common tokens
                if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
                    match = 'C'
                else:
                    match = 'c'
            else:
                match = ''
                
    if match == '':
        n_common = len(set(name_tokens).intersection(set(term_tokens)))
        if n_common != 0:
            match = n_common
        
#     print('MATCH', match)
#     if match != '':
#         print('MATCH', match)
#         print('TERM', term)
#         print('NAME', name)
#         print()
    return match

# failings
    # straight matches

In [7]:
# doing it this way so I can add 'print' to monitor progress
mark_data = []
for name in stir_fry_data['name']:
    print(name)
    mark_data.append([get_mark(name, term) for term in all_terms])

# term_name_marks = pd.DataFrame(mark_data, columns = all_terms)
# term_name_marks['name'] = pd.Series(stir_fry_data['name'].values.tolist())

# term_name_scores.to_csv(os.path.join(root_path, 'DATA/term_name_scores_common.csv'), index=False)

ALMONDS (and UNSWEETENED ALMOND BUTTER; see also MILK, ALMOND)
ANISE SEEDS
APPLES (and APPLE CIDER, APPLE JUICE and/or APPLESAUCE)
ARUGULA (aka ROCKET)
ASPARAGUS
BASIL
BEANS, BLACK (aka TURTLE BEANS)
BEANS, PINTO
BEETS
BELL PEPPERS—IN GENERAL, or MIXED
BOK CHOY (aka CHINESE CABBAGE or PAK CHOI)
BRAGG LIQUID AMINOS
BROCCOLI
BROCCOLINI
BRUSSELS SPROUTS
BUTTER
CABBAGE, GREEN
CABBAGE, RED
CARROTS
CASHEWS and CASHEW NUT BUTTER
CAULIFLOWER
CELERY
CHARD, e.g., RAINBOW, RED/RUBY, SWISS, or MIXED
CHEESE, CHEDDAR
CHEESE, PARMESAN
CHICKPEAS (aka GARBANZO BEANS)
CHILES, HABANERO
CHILES, JALAPEÑO
CHILI PEPPER FLAKES
CHILI POWDER
CHIVES
CILANTRO (aka CHINESE PARSLEY or FRESH CORIANDER LEAF)
COCONUT BUTTER
CORN
CORNMEAL and POLENTA (see also GRITS)
CUMIN
EGGPLANT (aka AUBERGINE)
FENNEL
FENNEL SEEDS
FLAXSEEDS
GARLIC
GINGER—IN GENERAL
GREENS, BEET
GREENS, DANDELION
HONEY—IN GENERAL
KALE
LEEKS
LEGUMES (see also specific BEANS, CHICKPEAS, LENTILS, PEANUTS, PEAS, and SOYBEANS)
LEMONS
LENTILS, BROWN
LENTIL

In [8]:
# doing it this way so I can add 'print' to monitor progress
# mark_data = []
# for name in salad_matches['name']:
#     print(name)
#     mark_data.append([get_mark(name, term) for term in all_terms])

term_name_marks = pd.DataFrame(mark_data, columns = all_terms)
term_name_marks['name'] = pd.Series(stir_fry_data['name'].values.tolist())

# term_name_scores.to_csv(os.path.join(root_path, 'DATA/term_name_scores_common.csv'), index=False)

In [9]:
# doing it this way so I can add 'print' to monitor progress
# mark_data = []
# for name in salad_matches['name']:
#     print(name)
#     mark_data.append([get_mark(name, term) for term in all_terms])

# term_name_marks = pd.DataFrame(mark_data, columns = all_terms)
# term_name_marks['name'] = pd.Series(stir_fry_data['name'].values.tolist())

term_name_marks.to_csv(os.path.join(root_path, 'DATA/stir_fry_term_name_marks.csv'), index=False)

In [70]:
for i, row in term_name_marks.iterrows():
    for val in row:
        if val != '':
            print(val)

y
ALMONDS (and UNSWEETENED ALMOND BUTTER; see also MILK, ALMOND)
y
y
y
y
y
Y
y
y
Y
y
y
y
y
y
y
Y
ANISE SEEDS
1.0
y
Y
Y
Y
y
y
y
y
Y
y
Y
y
y
c
1.0
y
c
y
Y
Y
y
y
y
Y
c
y
y
Y
c
Y
y
y
y
Y
1.0
1.0
c
y
y
APPLES (and APPLE CIDER, APPLE JUICE and/or APPLESAUCE)
d
y
Y
y
1.0
y
y
y
y
y
y
y
y
y
Y
d
y
d
y
d
y
y
y
c
d
ARTICHOKE HEARTS (see also ARTICHOKES)
y
Y
y
1.0
y
y
1
y
y
y
y
1
y
Y
y
y
y
y
y
c
ARTICHOKES, JERUSALEM (aka SUNCHOKES)
d
1.0
y
y
1.0
1
y
y
1
y
Y
y
y
Y
c
1
y
d
y
d
y
1
d
y
1
1.0
d
y
y
c
d
Y
d
y
y
Y
y
c
ARUGULA (aka ROCKET)
y
y
1
1
c
y
y
Y
y
1
1.0
1
1
c
1
y
1.0
1
c
y
1
1
y
d
1
1.0
1.0
ASPARAGUS
Y
y
Y
y
y
y
y
y
d
y
y
y
y
y
y
y
y
y
y
y
y
y
BASIL
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan

d
Y
y
Y
y
y
c
c
d
y
1
c
1.0
y
c
c
c
Y
Y
c
c
Y
y
Y
d
c
c
y
Y
d
d
c
d
CHILES, JALAPEÑO
Y
d
1.0
c
c
y
d
c
Y
y
c
1.0
y
c
y
d
Y
d
Y
Y
d
d
Y
d
y
c
c
Y
c
c
Y
1.0
d
Y
d
d
Y
y
Y
y
y
c
d
c
y
1
c
1.0
y
d
d
c
Y
Y
c
d
Y
y
Y
d
d
d
y
Y
d
c
c
d
CHILES, SERRANO
Y
c
1.0
c
d
y
d
c
Y
y
c
1.0
y
c
y
D
Y
d
Y
Y
c
d
Y
c
y
c
d
Y
d
c
Y
1.0
c
Y
c
D
Y
y
Y
y
y
d
d
D
y
1
c
1.0
y
d
c
c
Y
Y
c
d
Y
y
Y
d
D
d
y
Y
d
d
d
d
CHILI PEPPER FLAKES
Y
c
1.0
c
c
y
c
c
Y
y
c
1.0
y
c
y
c
Y
c
Y
Y
c
c
Y
c
y
c
c
Y
c
c
Y
1.0
c
Y
c
c
Y
y
Y
y
y
c
c
c
y
1
c
1.0
y
c
c
c
Y
Y
c
c
Y
y
Y
c
c
c
y
Y
c
c
c
c
CHILI POWDER
Y
c
1.0
c
c
y
c
c
Y
y
c
1.0
y
c
y
c
Y
c
Y
Y
c
c
Y
c
y
c
c
Y
c
c
Y
1.0
c
Y
c
c
Y
y
Y
y
y
c
c
c
y
1
c
1.0
y
c
c
c
Y
Y
c
c
Y
y
Y
c
c
c
y
Y
c
c
c
c
CHIVES
1
y
1.0
y
y
y
y
1
1.0
y
c
Y
1.0
Y
y
y
Y
c
y
y
y
1.0
1
y
1
y
y
y
Y
1.0
CHIVES, GARLIC (aka CHINESE CHIVES)
y
Y
y
y
1.0
d
y
y
y
y
y
Y
y
d
y
y
y
1.0
CILANTRO (aka CHINESE PARSLEY or FRESH CORIANDER LEAF)
y
y
y
y
y
y
y
COCONUT BUTTER
y
y
y
y
Y
y
y
y
y
y
y
y
COCONUT WATER
Y
y
y
Y
Y
y
y
y

# 2. Extracting "pairs with" data

In [8]:
# term_name_matches_raw = pd.read_csv(os.path.join(root_path, 'DATA/term_name_matches.csv'))
# term_name_matches = term_name_matches_raw.replace(['0', '1', '2', '3', '4', '5', 0, 1, 2, 3, 4, 5, float('nan')], '')
# term_name_matches_lower = term_name_matches.replace('Y', 'y')

In [9]:
# # create a dataframe with name and a list of pairing terms for each ingredient
# pairing_data = pd.DataFrame({
#     'name': salad_data['name'],
#     'pairs_with_terms': ingredient_pairs_with_terms
# })

# def get_pairs_with_names(row):
# #     print(row['name'], type(row['name']))
#     lower_names = []
#     upper_names = []
#     for term in row['pairs_with_terms']:
#         if term in term_name_matches.columns.values.tolist():
#             lower_names += term_name_matches[term_name_matches[term] == 'y']['name'].values.tolist()
#             upper_names += term_name_matches[term_name_matches[term] == 'Y']['name'].values.tolist()
#         else:
#             print('OH NO didnt find term:', term)
# #     for name in lower_names + upper_names:
# #         if name == row['name']:
# #             print('DUPLICATE')
# #     print(lower_names, upper_names)
#     row['lower_pairs_with_names'] = [lower_name for lower_name in lower_names if lower_name != row['name']]
#     row['upper_pairs_with_names'] = [upper_name for upper_name in upper_names if upper_name != row['name']]
#     row['all_pairs_with_names'] = lower_names + upper_names
#     return row

# pairing_data = pairing_data.apply(get_pairs_with_names, axis=1)

In [10]:
term_name_matches_raw = pd.read_csv(os.path.join(root_path, 'DATA/term_name_matches_specific.csv'))
term_name_matches = term_name_matches_raw.replace(['0', '1', '2', '3', '4', '5', 0, 1, 2, 3, 4, 5, float('nan')], '')

In [11]:
# # create a dataframe with name and a list of pairing terms for each ingredient
# pairing_data = pd.DataFrame({
#     'name': salad_data['name'],
#     'pairs_with_terms': ingredient_pairs_with_terms
# })

# def get_pairs_with_names(row):
# #     print(row['name'], type(row['name']))
#     lower_category_names = []
#     lower_direct_names = []
#     upper_category_names = []
#     upper_direct_names = []
#     for term in row['pairs_with_terms']:
#         if term in term_name_matches.columns.values.tolist():
#             lower_category_names += term_name_matches[term_name_matches[term] == 'c']['name'].values.tolist()
#             lower_direct_names += term_name_matches[term_name_matches[term] == 'd']['name'].values.tolist()
#             upper_category_names += term_name_matches[term_name_matches[term] == 'C']['name'].values.tolist()
#             upper_direct_names += term_name_matches[term_name_matches[term] == 'D']['name'].values.tolist()
#         else:
#             pass
# #             print('OH NO didnt find term:', term)
#     row['lower_category_pairs_with_names'] = list(set([lower_category_name for lower_category_name in lower_category_names if lower_category_name != row['name']]))
#     row['lower_direct_pairs_with_names'] = list(set([lower_direct_name for lower_direct_name in lower_direct_names if lower_direct_name != row['name']]))
#     row['upper_category_pairs_with_names'] = list(set([upper_category_name for upper_category_name in upper_category_names if upper_category_name != row['name']]))
#     row['upper_direct_pairs_with_names'] = list(set([upper_direct_name for upper_direct_name in upper_direct_names if upper_direct_name != row['name']]))
#     row['lower_pairs_with_names'] = list(set(row['lower_category_pairs_with_names'] + row['lower_direct_pairs_with_names']))
#     row['upper_pairs_with_names'] = list(set(row['upper_category_pairs_with_names'] + row['upper_direct_pairs_with_names']))
#     row['all_pairs_with_names'] = list(set(row['lower_pairs_with_names'] + row['upper_pairs_with_names']))
    
#     row['lc_sorted_pairs'] = tuple(sorted(row['lower_category_pairs_with_names']))
#     row['ld_sorted_pairs'] = tuple(sorted(row['lower_direct_pairs_with_names']))
#     row['uc_sorted_pairs'] = tuple(sorted(row['upper_category_pairs_with_names']))
#     row['ud_sorted_pairs'] = tuple(sorted(row['upper_direct_pairs_with_names']))
#     row['l_sorted_pairs'] = tuple(sorted(row['lower_pairs_with_names']))
#     row['u_sorted_pairs'] = tuple(sorted(row['upper_pairs_with_names']))
#     row['a_sorted_pairs'] = tuple(sorted(row['all_pairs_with_names']))
#     return row

# pairing_data = pairing_data.apply(get_pairs_with_names, axis=1)

In [12]:
# CREATE PAIRING DATA MATRIX (names x names)
# takes a few minutes

pairing_data = pd.DataFrame({
    'name': salad_data['name'],
    'pairs_with_terms': ingredient_pairs_with_terms
})

for name in salad_data['name']:
    pairing_data[name] = pd.Series(['']*len(salad_data['name']))

def get_pairs_with_names(row):
#     print(row['name'], type(row['name']))
    lower_category_names = []
    lower_direct_names = []
    upper_category_names = []
    upper_direct_names = []
    for term in row['pairs_with_terms']:
        if term in term_name_matches.columns.values.tolist():
            lower_category_names += term_name_matches[term_name_matches[term] == 'c']['name'].values.tolist()
            lower_direct_names += term_name_matches[term_name_matches[term] == 'd']['name'].values.tolist()
            upper_category_names += term_name_matches[term_name_matches[term] == 'C']['name'].values.tolist()
            upper_direct_names += term_name_matches[term_name_matches[term] == 'D']['name'].values.tolist()
        else:
            pass
#             print('OH NO didnt find term:', term)
    
    for lower_category_name in lower_category_names:
        row[lower_category_name] = 'c'
    for lower_direct_name in lower_direct_names:
        row[lower_direct_name] = 'd'
    for upper_category_name in upper_category_names:
        row[upper_category_name] = 'C'
    for upper_direct_name in upper_direct_names:
        row[upper_direct_name] = 'D'
    
#     row['lower_category_pairs_with_names'] = list(set([lower_category_name for lower_category_name in lower_category_names if lower_category_name != row['name']]))
#     row['lower_direct_pairs_with_names'] = list(set([lower_direct_name for lower_direct_name in lower_direct_names if lower_direct_name != row['name']]))
#     row['upper_category_pairs_with_names'] = list(set([upper_category_name for upper_category_name in upper_category_names if upper_category_name != row['name']]))
#     row['upper_direct_pairs_with_names'] = list(set([upper_direct_name for upper_direct_name in upper_direct_names if upper_direct_name != row['name']]))
#     row['lower_pairs_with_names'] = list(set(row['lower_category_pairs_with_names'] + row['lower_direct_pairs_with_names']))
#     row['upper_pairs_with_names'] = list(set(row['upper_category_pairs_with_names'] + row['upper_direct_pairs_with_names']))
#     row['all_pairs_with_names'] = list(set(row['lower_pairs_with_names'] + row['upper_pairs_with_names']))
    
#     row['lc_sorted_pairs'] = tuple(sorted(row['lower_category_pairs_with_names']))
#     row['ld_sorted_pairs'] = tuple(sorted(row['lower_direct_pairs_with_names']))
#     row['uc_sorted_pairs'] = tuple(sorted(row['upper_category_pairs_with_names']))
#     row['ud_sorted_pairs'] = tuple(sorted(row['upper_direct_pairs_with_names']))
#     row['l_sorted_pairs'] = tuple(sorted(row['lower_pairs_with_names']))
#     row['u_sorted_pairs'] = tuple(sorted(row['upper_pairs_with_names']))
#     row['a_sorted_pairs'] = tuple(sorted(row['all_pairs_with_names']))
    return row

pairing_data = pairing_data.apply(get_pairs_with_names, axis=1)
pairing_data.replace(float('nan'), '', inplace=True)

In [13]:
# SYNC PAIRING DATA MATRIX (make sure [a][b] agrees with [b][a])

for index_1, name_1 in enumerate(pairing_data['name'].values.tolist()):
    for index_2, name_2 in enumerate(pairing_data['name'].values.tolist()):
        value_1 = pairing_data[name_1][index_2]
        value_2 = pairing_data[name_2][index_1]
        
        if name_1 == name_2:
            proper_value = ''
        elif value_1 == 'D' or value_2 == 'D':
            proper_value = 'D'
        elif value_1 == 'C' or value_2 == 'C':
            proper_value = 'C'
        elif value_1 == 'd' or value_2 == 'd':
            proper_value = 'd'
        elif value_1 == 'c' or value_2 == 'c':
            proper_value = 'c'
        else:
            proper_value = ''
        
        pairing_data[name_1][index_2] = proper_value
        pairing_data[name_2][index_1] = proper_value

In [14]:
# REPRESENT PAIRING DATA AS LISTS

def get_pairs_with_names(row):
    lower_category_name = pairing_data[pairing_data[row['name']] == 'c']['name'].values.tolist()
    lower_direct_name = pairing_data[pairing_data[row['name']] == 'd']['name'].values.tolist()
    upper_category_name = pairing_data[pairing_data[row['name']] == 'C']['name'].values.tolist()
    upper_direct_name = pairing_data[pairing_data[row['name']] == 'D']['name'].values.tolist()
    
    row['lower_category_names'] = pairing_data[pairing_data[row['name']] == 'c']['name'].values.tolist()
    row['lower_direct_names'] = pairing_data[pairing_data[row['name']] == 'd']['name'].values.tolist()
    row['upper_category_names'] = pairing_data[pairing_data[row['name']] == 'C']['name'].values.tolist()
    row['upper_direct_names'] = pairing_data[pairing_data[row['name']] == 'D']['name'].values.tolist()
    row['lower_names'] = row['lower_category_names'] + row['lower_direct_names']
    row['upper_names'] = row['upper_category_names'] + row['upper_direct_names']
    row['all_names'] = row['lower_names'] + row['upper_names']
    
    return row

pairing_data = pairing_data.apply(get_pairs_with_names, axis=1)

In [15]:
# pairing_data.to_csv(os.path.join(root_path, 'DATA/pairing_data.csv'), index=False)
# pairing_data = pd.read_csv(os.path.join(root_path, 'DATA/pairing_data.csv'))

# 3. Creating "clashes with" data

In [16]:
# name_name_clashes_blank = salad_data[['name', 'protein_cheese_sub', 'salad_allium', 'fruit', 'veg']].copy()
# names = salad_data['name'].values.tolist()

# for i, col_name in enumerate(names):
#     name_name_clashes_blank[col_name] = pd.Series(['x']*(i+1) + ['']*(len(names)-(i+1)))

# name_name_clashes_blank.to_csv(os.path.join(root_path, 'DATA/name_name_clashes_blank.csv'), index=False)

In [17]:
name_name_clashes_input = pd.read_csv(os.path.join(root_path, 'DATA/name_name_clashes_input.csv'))
name_name_clashes_input = name_name_clashes_input.replace([float('nan'), 'x'], '')
columns = ['name'] + name_name_clashes_input['name'].values.tolist()
name_name_clashes_input = name_name_clashes_input[columns]

def get_clashes_with_data(row):
    data = pd.Series([])
#     print(row.index.values.tolist())
    name = row['name']
    data['name'] = name
    
    lower_names = name_name_clashes_input['name'][name_name_clashes_input[name] == 'y'].values.tolist()
    upper_names = name_name_clashes_input['name'][name_name_clashes_input[name] == 'Y'].values.tolist()
    if 'y' in lower_names:
        print(lower_names)
    
    for name in name_name_clashes_input['name']:
#         if name in ['y', 'Y']:
#             print(name)
        if row[name] == 'y':
            lower_names.append(name)
        elif row[name] == 'Y':
            upper_names.append(name)
    
    lower_names = list(set(lower_names))
    upper_names = list(set(upper_names))
    
    data['lower_clashes_with_names'] = lower_names
    data['upper_clashes_with_names'] = upper_names
    data['all_clashes_with_names'] = list(set(lower_names + upper_names)) # shouldn't be overlap here, but hey
    
    data['lower_clashes_with_pairs'] = [tuple(sorted([name, lower_name])) for lower_name in data['lower_clashes_with_names']]
    data['upper_clashes_with_pairs'] = [tuple(sorted([name, lower_name])) for lower_name in data['upper_clashes_with_names']]
    data['all_clashes_with_pairs'] = [tuple(sorted([name, lower_name])) for lower_name in data['all_clashes_with_names']]
    
    return data

clashes_with_data = name_name_clashes_input.apply(get_clashes_with_data, axis=1)
            
# # for row in name_name_clashes_input.iterrows():
# clashes_with_data = pd.DataFrame({'name': salad_data['name']})
# for i, row in clashes_with_data.iterrows():
# #     print(row, type(row))
#     name = row['name']
#     row['lower_clashes_with'] = name_name_clashes_input[name][name_name_clashes_input[name] == 'y']
#     row['upper_clashes_with'] = name_name_clashes_input[name][name_name_clashes_input[name] == 'Y']
#     row['all_clashes_with'] = row['lower_clashes_with'] + row['upper_clashes_with']

In [18]:
all_lower_clashes_with_pairs = list(set(clashes_with_data['lower_clashes_with_pairs'].sum()))
all_upper_clashes_with_pairs = list(set(clashes_with_data['upper_clashes_with_pairs'].sum()))
all_clashes_with_pairs = list(set(clashes_with_data['all_clashes_with_pairs'].sum()))

# 4. Salad recipe generators

In [19]:
salad_greens = salad_data[salad_data['salad_green'] == 'y']

salad_extras = salad_data[salad_data['salad_extra'] == 'y']
salad_extra_veg = salad_data[(salad_data['veg'] == 'y') & (salad_data['salad_extra'] == 'y')]
salad_extra_fruits = salad_data[(salad_data['fruit'] == 'y') & (salad_data['salad_extra'] == 'y')]
salad_extra_nuts = salad_data[(salad_data['protein_seed'] == 'y') & (salad_data['salad_extra'] == 'y')]
salad_extra_seeds = salad_data[(salad_data['protein_nut'] == 'y') & (salad_data['salad_extra'] == 'y')]
salad_extra_tomatoes = salad_data[salad_data['salad_extra_tomato'] == 'y']
salad_extra_olives = salad_data[salad_data['salad_extra_olive'] == 'y']
salad_extra_cheeses = salad_data[salad_data['salad_extra_cheese'] == 'y']
salad_extra_eggs = salad_data[salad_data['salad_extra_egg'] == 'y']
salad_extra_croutons = salad_data[salad_data['salad_extra_crouton'] == 'y']

salad_dressing_oils = salad_data[salad_data['salad_dressing_oil'] == 'y']
salad_dressing_vinegars = salad_data[salad_data['salad_dressing_vinegar'] == 'y']
salad_dressing_salts = salad_data[salad_data['salad_dressing_salt'] == 'y']
salad_dressing_peppers = salad_data[salad_data['salad_dressing_pepper'] == 'y']
salad_dressing_garlics = salad_data[salad_data['salad_dressing_garlic'] == 'y']

## 4.1. Best of many

### 4.1.1. Generator

In [26]:
# !pip install networkx
# !pip install pyvis

import networkx as nx
from pyvis import network as net

In [66]:
#IDEAS
    # savory v sweet templates (avoid eggs/mushroom/hard veg w fruit...)
    # get eg working, weight it as 2
    # start out w 1 base salad, extra (randomly selected from vip list), then branch out
    # control for eg lots of types of citrus, alliums
    # mark strong flavors, treat them separately
    # select main ingredients from each category that go with each other, then branch out from each, weighting traditionally at the end
    # work off of pairs_with data for categories, while at same time picking categories then within categories (to account for eg allium bias)
    # *only* match eg.s with specifically named; also, consider not matching categories the weird way
    # add a bonus if category of ingredient pairs well with other ingredients in salad (meh)
    # add in dressing garlics, when applicable
    # CONSIDER
        # replacing average net dist with something like, edges/possible edges

# TODO
    # add dried cranberries to salad stuffs?
    # maybe some other dried fruit? maybe not.
    
# UI PRIORITIES
    # speed
    # translate names to common (e.g. ALLMONDS (AND UNSWEETENED ALMOND BUTTER) -> Almonds, sliced)
    # maybe consisder one of those spokes-of-wheel charts to display stats/scores
    
# RECIPE DETAILS
    # maybe suggest diluting e.g. sesame oil w regular olive oil
    
# ==================================================================================================================
# ==================================================================================================================

# ALGORITHM A (best of many)
top_score = 0
for i in range(300):
    n_subgraphs = 2
    while n_subgraphs > 1: # keep shuffling until you get a well connected graph
        n_greens = random.randrange(2, 4)
        n_extras = random.randrange(2, 6)
        n_dressing_oils = 1
        n_dressing_vinegars = 1
        n_dressing_salts = 1
        n_dressing_peppers = 1
        # n_dressing_garlics = random.randrange(0, 2) # maybe make presence dependent on the rest. or, just leave out for now.
        
        selected_greens = salad_greens.sample(n_greens)
        selected_extras = salad_extras.sample(n_extras)
        selected_dressing_oils = salad_dressing_oils.sample(n_dressing_oils)
        selected_dressing_vinegars = salad_dressing_vinegars.sample(n_dressing_vinegars)
        selected_dressing_salts = salad_dressing_salts.sample(n_dressing_salts)
        selected_dressing_peppers = salad_dressing_peppers.sample(n_dressing_peppers)
        selected_ingredients = selected_greens.append(selected_extras).append(selected_dressing_oils).append(selected_dressing_vinegars).append(selected_dressing_salts).append(selected_dressing_peppers)

        lower_category_pairs = []
        lower_direct_pairs = []
        upper_category_pairs = []
        upper_direct_pairs = []
        ingredients_list = selected_ingredients['name'].values.tolist()
        already_checked = []
        for ingredient_name in ingredients_list:
            for lc_name in pairing_data['lower_category_names'][pairing_data['name'] == ingredient_name].iloc[0]:
                if lc_name in ingredients_list and not lc_name in already_checked:
                    lower_category_pairs.append([ingredient_name, lc_name])
            for ld_name in pairing_data['lower_direct_names'][pairing_data['name'] == ingredient_name].iloc[0]:
                if ld_name in ingredients_list and not ld_name in already_checked:
                    lower_direct_pairs.append([ingredient_name, ld_name])
            for uc_name in pairing_data['upper_category_names'][pairing_data['name'] == ingredient_name].iloc[0]:
                if uc_name in ingredients_list and not uc_name in already_checked:
                    upper_category_pairs.append([ingredient_name, uc_name])
            for ud_name in pairing_data['upper_direct_names'][pairing_data['name'] == ingredient_name].iloc[0]:
                if ud_name in ingredients_list and not ud_name in already_checked:
                    upper_direct_pairs.append([ingredient_name, ud_name])
            already_checked.append(ingredient_name)

        lower_pairs = lower_category_pairs + lower_direct_pairs
        upper_pairs = upper_category_pairs + upper_direct_pairs
        all_pairs = lower_pairs + upper_pairs
        
#         all_pairs_sp = [tuple(sorted(pair)) for pair in all_pairs]
#         print(len(all_pairs_sp), len(list(set(all_pairs_sp))))

#         print('INGREDIENTS', ingredients_list)
#         print()
#         print('LC PAIRS', lower_category_pairs)
#         print()
#         print()

        G = nx.Graph()
        G.add_nodes_from(selected_ingredients['name'].values.tolist())
        G.add_edges_from(all_pairs)
        n_subgraphs = len(list(nx.connected_component_subgraphs(G)))
            
    score = 0

# SIMPLER ALTERNATIVE: bonus for proportion of actual pairs over possible pairs? could then combine with pair strength bonus?
# PAIRING DENSITY BONUS ============================================================================================
    # ranges from roughly (.1 to 1) * 4
    average_shortest_path_length = nx.average_shortest_path_length(G)
    average_shortest_path_score = 2.5 / average_shortest_path_length - 1.1
#     print(average_shortest_path_score)
    score += average_shortest_path_score * 4    
    
# # UPPER PAIRING BONUS ==============================================================================================
#     # ranges from roughly (.25 to 1) * 3
#     upper_proportion_score = len(upper_pairs) / len(all_pairs) * 2.5 # messed with this, not sure if it still works
# #     print(upper_proportion_score)
# #     print(len(upper_pairs), len(lower_pairs), len(all_pairs))
# #     print()
#     score += upper_proportion_score * 3

# PAIR STRENGTH BONUS ==============================================================================================
    # ranges from roughly (0 to 1) * 3

    # I'm thinking of 'lower category' as default, and awarding points for steps up from that
    ld_bonus = 1*len(lower_direct_pairs)
    uc_bonus = 2*len(upper_category_pairs)
    ud_bonus = 5*len(upper_direct_pairs)
    pair_strength_score = .6*(ld_bonus + uc_bonus + ud_bonus)/len(all_pairs) # otherwise would tend toward large salads
#     print('LC', len(lower_category_pairs))
#     print('LD', len(lower_direct_pairs))
#     print('UC', len(upper_category_pairs))
#     print('UD', len(upper_direct_pairs))
#     print('SCORE', pair_strength_score)
#     print()
    score += pair_strength_score * 3

# important but easy to avoid, so not weighted too heavily
# CLASH PENALTY ====================================================================================================
    # ranges from roughly (0 to 1) * -1.5
    all_clashing_pairs = []
    selected_ingredients_list = selected_ingredients['name'].values.tolist()
    for name in selected_ingredients_list:
        names_that_clash_with_name = clashes_with_data['all_clashes_with_names'][clashes_with_data['name'] == name].iloc[0]
        all_clashing_names = set(selected_ingredients_list).intersection(set(names_that_clash_with_name)) # selected names that clash with this selected name
        all_clashing_pairs += [tuple(sorted([name, all_clashing_name])) for all_clashing_name in all_clashing_names]

    all_clashing_pairs = list(set(all_clashing_pairs))
    all_clashing_pairs_score = len(list(all_clashing_pairs)) / 4
    score += len(all_clashing_pairs) * -1.5

# # FRUIT BONUS ======================================================================================================
#     # ranges from roughly (0 to 3) * .1
#     n_fruit = len(selected_ingredients[selected_ingredients['fruit'] == 'y'])
#     score += n_fruit * .1
    
# # NUT SEED BONUS ===================================================================================================    
#     # ranges from roughly (0 to 2) * .15
#     n_nut_seed = len(selected_ingredients[selected_ingredients['protein_nut_seed'] == 'y'])
#     score += n_nut_seed * .15

# FLAVOR BALANCE BONUS =============================================================================================
    # ranges from roughly (0 to 1) * 1
    n_sweet_lower = len(selected_ingredients[selected_ingredients['sweet'] == 'y'])
    n_sweet_upper = len(selected_ingredients[selected_ingredients['sweet'] == 'Y'])
    n_salty_lower = len(selected_ingredients[selected_ingredients['salty'] == 'y'])
    n_salty_upper = len(selected_ingredients[selected_ingredients['salty'] == 'Y'])
    n_sour_lower = len(selected_ingredients[selected_ingredients['sour'] == 'y'])
    n_sour_upper = len(selected_ingredients[selected_ingredients['sour'] == 'Y'])
    n_savory_lower = len(selected_ingredients[selected_ingredients['savory'] == 'y'])
    n_savory_upper = len(selected_ingredients[selected_ingredients['savory'] == 'Y'])
    n_bitter_lower = len(selected_ingredients[selected_ingredients['bitter'] == 'y'])
    n_bitter_upper = len(selected_ingredients[selected_ingredients['bitter'] == 'Y'])
    n_spicy_lower = len(selected_ingredients[selected_ingredients['spicy'] == 'y'])
    n_spicy_upper = len(selected_ingredients[selected_ingredients['spicy'] == 'Y'])
    
    # each varies from roughly .5 to 1
    sweet_score = (n_sweet_lower/2 + n_sweet_upper)/5
    salty_score = (n_salty_lower/2 + n_salty_upper)*2/5
    sour_score = (n_sour_lower/2 + n_sour_upper)*2/5
    savory_score = (n_savory_lower/2 + n_savory_upper)*3/5
    bitter_score = (n_bitter_lower/2 + n_bitter_upper)*3/5
    spicy_score = (n_spicy_lower/2 + n_spicy_upper)*2/5
    
    flavor_balance_score = 5 / (1 + abs(1-sweet_score) + abs(1-salty_score) + abs(1-sour_score) + abs(1-savory_score) + abs(1-spicy_score)) - 1.25
#     print(flavor_balance_score)
    
    score += flavor_balance_score
#     print(sweet_score, salty_score, sour_score, savory_score, bitter_score, spicy_score)
#     print()
    
#     print(n_sweet_lower, n_sweet_upper)
#     print(n_salty_lower, n_salty_upper)
#     print(n_sour_lower, n_sour_upper)
#     print(n_savory_lower, n_savory_upper)
#     print(n_bitter_lower, n_bitter_upper)
#     print(n_spicy_lower, n_spicy_upper)
#     print()

# TEXTURE BALANCE BONUS ============================================================================================
    # ranges from roughly (0 to 1) * .75
    n_crunchy_lower = len(selected_ingredients[selected_ingredients['salad_crunchy'] == 'y'])
    n_crunchy_upper = len(selected_ingredients[selected_ingredients['salad_crunchy'] == 'Y'])
    n_chewy_lower = len(selected_ingredients[selected_ingredients['salad_chewy'] == 'y'])
    n_chewy_upper = len(selected_ingredients[selected_ingredients['salad_chewy'] == 'Y'])
    n_juicy_lower = len(selected_ingredients[selected_ingredients['salad_juicy'] == 'y'])
    n_juicy_upper = len(selected_ingredients[selected_ingredients['salad_juicy'] == 'Y'])
    
    # each ranges from roughly 0 to 1
    crunchy_score = (n_crunchy_lower/2 + n_crunchy_upper)/3
    chewy_score = (n_chewy_lower/2 + n_chewy_upper)
    juicy_score = (n_juicy_lower/2 + n_juicy_upper)/3
#     print(crunchy_score, chewy_score, juicy_score)
    
    texture_balance_score = 4 / (1 + abs(1-crunchy_score) + abs(1-chewy_score) + abs(1-juicy_score)) - 1
#     print(texture_balance_score)
#     print()
    
    score += texture_balance_score * .75

# seems like it's hard to balance food groups on top of everything else. pity the scores aren't more independent
# FOOD GROUP BALANCE BONUS =========================================================================================
    # ranges from roughly (25 to 1) * 2
    n_fruit = len(selected_ingredients[selected_ingredients['fruit'] == 'y'])
    n_veg = len(selected_ingredients[selected_ingredients['veg'] == 'y'])
    n_protein = len(selected_ingredients[selected_ingredients['protein'] == 'y'])
    
    # each varies from roughly 0 to 1 (sometimes a little over)
    fruit_score = n_fruit / 3
    veg_score = n_veg / 5
    protein_score = n_protein / 3
#     print(fruit_score, veg_score, protein_score)
    
    food_group_balance_score = 3 / (1 + abs(1-fruit_score) + abs(1-veg_score) + abs(1-protein_score)) - .75
#     print(food_group_balance_score)
#     print()
    
    score += food_group_balance_score * 2
    
    if score > top_score:
        top_score = score
        top_food_group_balance_score = food_group_balance_score
        top_average_shortest_path_score = average_shortest_path_score
        top_flavor_balance_score = flavor_balance_score
        top_texture_balance_score = texture_balance_score
        top_all_clashing_pairs_score = all_clashing_pairs_score
        top_pair_strength_score = pair_strength_score
#         top_upper_pairs = upper_pairs
#         top_lower_pairs = lower_pairs
        top_lc_pairs = lower_category_pairs
        top_ld_pairs = lower_direct_pairs
        top_uc_pairs = upper_category_pairs
        top_ud_pairs = upper_direct_pairs
        top_selected_ingredients = selected_ingredients
        top_average_shortest_path_length = average_shortest_path_length
#         top_upper_proportion = len(upper_pairs) / (len(upper_pairs) + len(lower_pairs))
# print('TOP AVG SHORTEST PATH LENGTH', top_average_shortest_path_length)
# print('TOP UPPER PROPORTION', top_upper_proportion)
print('TOP AVERAGE SHORTEST PATH SCORE', top_average_shortest_path_score * 4)
print('TOP PAIR STRENGTH SCORE', top_pair_strength_score * 3)
print('TOP ALL CLASHING PAIRS SCORE', top_all_clashing_pairs_score * -1.5)
print('TOP FLAVOR BALANCE SCORE', top_flavor_balance_score)
print('TOP TEXTURE BALANCE SCORE', top_texture_balance_score * .75)
print('TOP FOOD GROUP BALANCE SCORE', top_food_group_balance_score * 2)
print('TOP_SCORE', top_score)            
    
top_net = net.Network(notebook=True)

nodes = top_selected_ingredients['name'].tolist()

def get_color(row):
#     print(type(row))
#     print(row)
    if row['salad_green'] == 'y':
        return 'lightgreen'
    elif row['salad_extra'] == 'y':
        if row['veg'] == 'y':
            return 'green'
        elif row['fruit'] == 'y':
            return 'orange'
        elif row['protein_nut_seed'] == 'y':
            return 'brown'
        else:
            return 'lightblue'
    elif row['salad_dressing'] == 'y':
        return 'lightgrey'
    
nodes_color = top_selected_ingredients.apply(get_color, axis=1).tolist()

top_net.add_nodes(
    nodes=nodes,
    color=nodes_color
)

for pair in top_lc_pairs:
    top_net.add_edge(pair[0], pair[1], physics=False, color='lightgrey')

for pair in top_ld_pairs:
    top_net.add_edge(pair[0], pair[1], physics=False, color='grey')
    
for pair in top_uc_pairs:
    top_net.add_edge(pair[0], pair[1], color='darkgrey')
    
for pair in top_ud_pairs:
    top_net.add_edge(pair[0], pair[1], color='black')

vegan = top_selected_ingredients['not_vegan'].sum() == ''
gluten_free = top_selected_ingredients['gluten'].sum() == ''
print(vegan, gluten_free)

top_net.show('top_net.html')

TOP AVERAGE SHORTEST PATH SCORE 3.2744186046511627
TOP PAIR STRENGTH SCORE 3.4434782608695653
TOP ALL CLASHING PAIRS SCORE -0.0
TOP FLAVOR BALANCE SCORE 0.4166666666666665
TOP TEXTURE BALANCE SCORE 0.37499999999999983
TOP FOOD GROUP BALANCE SCORE 1.5
TOP_SCORE 9.009563532187395
False True


### 4.1.2 Recording recipes

In [19]:
try:
    salad_recipe_data = pd.read_csv(os.path.join(root_path, 'DATA/salad_recipe_data_latest.csv'))
    print('SALAD RECIPE DATA IMPORT SUCCESSFUL')
except:
    salad_recipe_data = pd.DataFrame({
        'vegan': [],
        'gluten_free': [],
        'basic': [],
        'best_of': [],
        'score': [],
        'pairing_density_bonus': [],
        'pair_strength_bonus': [],
        'clash_penalty': [],
        'flavor_balance_bonus': [],
        'texture_balance_bonus': [],
        'food_group_balance_bonus': [],
        'lc_pairs': [],
        'ld_pairs': [],
        'uc_pairs': [],
        'ud_pairs':[],
        'clashing_pairs': [],
        'ingredient_names': [],
        'leafy_green_names': [],
        'extra_names': [],
        'dressing_names': [],
    })
    print('IMPORT FAILED, CREATING NEW SALAD RECIPE DATAFRAME')

SALAD RECIPE DATA IMPORT SUCCESSFUL


In [23]:
salad_greens = salad_data[salad_data['salad_green'] == 'y']

salad_extras = salad_data[salad_data['salad_extra'] == 'y']
salad_extra_veg = salad_data[(salad_data['veg'] == 'y') & (salad_data['salad_extra'] == 'y')]
salad_extra_fruits = salad_data[(salad_data['fruit'] == 'y') & (salad_data['salad_extra'] == 'y')]
salad_extra_nuts = salad_data[(salad_data['protein_seed'] == 'y') & (salad_data['salad_extra'] == 'y')]
salad_extra_seeds = salad_data[(salad_data['protein_nut'] == 'y') & (salad_data['salad_extra'] == 'y')]
salad_extra_tomatoes = salad_data[salad_data['salad_extra_tomato'] == 'y']
salad_extra_olives = salad_data[salad_data['salad_extra_olive'] == 'y']
salad_extra_cheeses = salad_data[salad_data['salad_extra_cheese'] == 'y']
salad_extra_eggs = salad_data[salad_data['salad_extra_egg'] == 'y']
salad_extra_croutons = salad_data[salad_data['salad_extra_crouton'] == 'y']

salad_dressing_oils = salad_data[salad_data['salad_dressing_oil'] == 'y']
salad_dressing_vinegars = salad_data[salad_data['salad_dressing_vinegar'] == 'y']
salad_dressing_salts = salad_data[salad_data['salad_dressing_salt'] == 'y']
salad_dressing_peppers = salad_data[salad_data['salad_dressing_pepper'] == 'y']
salad_dressing_garlics = salad_data[salad_data['salad_dressing_garlic'] == 'y']

In [24]:
# !pip install networkx

import networkx as nx

In [25]:
best_of = 300
for recipe_count in range(100):
    top_score = 0
    for i in range(best_of):
        n_subgraphs = 2
        while n_subgraphs > 1: # keep shuffling until you get a well connected graph
            n_greens = random.randrange(2, 4)
            n_extras = random.randrange(2, 6)
            n_dressing_oils = 1
            n_dressing_vinegars = 1
            n_dressing_salts = 1
            n_dressing_peppers = 1
            # n_dressing_garlics = random.randrange(0, 2) # maybe make presence dependent on the rest. or, just leave out for now.

            selected_greens = salad_greens.sample(n_greens)
            selected_extras = salad_extras.sample(n_extras)
            selected_dressing_oils = salad_dressing_oils.sample(n_dressing_oils)
            selected_dressing_vinegars = salad_dressing_vinegars.sample(n_dressing_vinegars)
            selected_dressing_salts = salad_dressing_salts.sample(n_dressing_salts)
            selected_dressing_peppers = salad_dressing_peppers.sample(n_dressing_peppers)
            selected_ingredients = selected_greens.append(selected_extras).append(selected_dressing_oils).append(selected_dressing_vinegars).append(selected_dressing_salts).append(selected_dressing_peppers)

            lower_category_pairs = []
            lower_direct_pairs = []
            upper_category_pairs = []
            upper_direct_pairs = []
            ingredients_list = selected_ingredients['name'].values.tolist()
            already_checked = []
            for ingredient_name in ingredients_list:
                for lc_name in pairing_data['lower_category_names'][pairing_data['name'] == ingredient_name].iloc[0]:
                    if lc_name in ingredients_list and not lc_name in already_checked:
                        lower_category_pairs.append([ingredient_name, lc_name])
                for ld_name in pairing_data['lower_direct_names'][pairing_data['name'] == ingredient_name].iloc[0]:
                    if ld_name in ingredients_list and not ld_name in already_checked:
                        lower_direct_pairs.append([ingredient_name, ld_name])
                for uc_name in pairing_data['upper_category_names'][pairing_data['name'] == ingredient_name].iloc[0]:
                    if uc_name in ingredients_list and not uc_name in already_checked:
                        upper_category_pairs.append([ingredient_name, uc_name])
                for ud_name in pairing_data['upper_direct_names'][pairing_data['name'] == ingredient_name].iloc[0]:
                    if ud_name in ingredients_list and not ud_name in already_checked:
                        upper_direct_pairs.append([ingredient_name, ud_name])
                already_checked.append(ingredient_name)

            lower_pairs = lower_category_pairs + lower_direct_pairs
            upper_pairs = upper_category_pairs + upper_direct_pairs
            all_pairs = lower_pairs + upper_pairs

    #         all_pairs_sp = [tuple(sorted(pair)) for pair in all_pairs]
    #         print(len(all_pairs_sp), len(list(set(all_pairs_sp))))

    #         print('INGREDIENTS', ingredients_list)
    #         print()
    #         print('LC PAIRS', lower_category_pairs)
    #         print()
    #         print()

            G = nx.Graph()
            G.add_nodes_from(selected_ingredients['name'].values.tolist())
            G.add_edges_from(all_pairs)
            n_subgraphs = len(list(nx.connected_component_subgraphs(G)))

        score = 0

    # SIMPLER ALTERNATIVE: bonus for proportion of actual pairs over possible pairs? could then combine with pair strength bonus?
    # PAIRING DENSITY BONUS ============================================================================================
        # ranges from roughly (.1 to 1) * 4
        average_shortest_path_length = nx.average_shortest_path_length(G)
        average_shortest_path_score = 2.5 / average_shortest_path_length - 1.1
    #     print(average_shortest_path_score)
        score += average_shortest_path_score * 4    

    # # UPPER PAIRING BONUS ==============================================================================================
    #     # ranges from roughly (.25 to 1) * 3
    #     upper_proportion_score = len(upper_pairs) / len(all_pairs) * 2.5 # messed with this, not sure if it still works
    # #     print(upper_proportion_score)
    # #     print(len(upper_pairs), len(lower_pairs), len(all_pairs))
    # #     print()
    #     score += upper_proportion_score * 3

    # PAIR STRENGTH BONUS ==============================================================================================
        # ranges from roughly (0 to 1) * 3

        # I'm thinking of 'lower category' as default, and awarding points for steps up from that
        ld_bonus = 1*len(lower_direct_pairs)
        uc_bonus = 2*len(upper_category_pairs)
        ud_bonus = 5*len(upper_direct_pairs)
        pair_strength_score = .6*(ld_bonus + uc_bonus + ud_bonus)/len(all_pairs) # otherwise would tend toward large salads
    #     print('LC', len(lower_category_pairs))
    #     print('LD', len(lower_direct_pairs))
    #     print('UC', len(upper_category_pairs))
    #     print('UD', len(upper_direct_pairs))
    #     print('SCORE', pair_strength_score)
    #     print()
        score += pair_strength_score * 3

    # important but easy to avoid, so not weighted too heavily
    # CLASH PENALTY ====================================================================================================
        # ranges from roughly (0 to 1) * -1.5
        all_clashing_pairs = []
        selected_ingredients_list = selected_ingredients['name'].values.tolist()
        for name in selected_ingredients_list:
            names_that_clash_with_name = clashes_with_data['all_clashes_with_names'][clashes_with_data['name'] == name].iloc[0]
            all_clashing_names = set(selected_ingredients_list).intersection(set(names_that_clash_with_name)) # selected names that clash with this selected name
            all_clashing_pairs += [tuple(sorted([name, all_clashing_name])) for all_clashing_name in all_clashing_names]

        all_clashing_pairs = list(set(all_clashing_pairs))
        all_clashing_pairs_score = len(list(all_clashing_pairs)) / 4
        score += len(all_clashing_pairs) * -1.5

    # # FRUIT BONUS ======================================================================================================
    #     # ranges from roughly (0 to 3) * .1
    #     n_fruit = len(selected_ingredients[selected_ingredients['fruit'] == 'y'])
    #     score += n_fruit * .1

    # # NUT SEED BONUS ===================================================================================================    
    #     # ranges from roughly (0 to 2) * .15
    #     n_nut_seed = len(selected_ingredients[selected_ingredients['protein_nut_seed'] == 'y'])
    #     score += n_nut_seed * .15

    # FLAVOR BALANCE BONUS =============================================================================================
        # ranges from roughly (0 to 1) * 1
        n_sweet_lower = len(selected_ingredients[selected_ingredients['sweet'] == 'y'])
        n_sweet_upper = len(selected_ingredients[selected_ingredients['sweet'] == 'Y'])
        n_salty_lower = len(selected_ingredients[selected_ingredients['salty'] == 'y'])
        n_salty_upper = len(selected_ingredients[selected_ingredients['salty'] == 'Y'])
        n_sour_lower = len(selected_ingredients[selected_ingredients['sour'] == 'y'])
        n_sour_upper = len(selected_ingredients[selected_ingredients['sour'] == 'Y'])
        n_savory_lower = len(selected_ingredients[selected_ingredients['savory'] == 'y'])
        n_savory_upper = len(selected_ingredients[selected_ingredients['savory'] == 'Y'])
        n_bitter_lower = len(selected_ingredients[selected_ingredients['bitter'] == 'y'])
        n_bitter_upper = len(selected_ingredients[selected_ingredients['bitter'] == 'Y'])
        n_spicy_lower = len(selected_ingredients[selected_ingredients['spicy'] == 'y'])
        n_spicy_upper = len(selected_ingredients[selected_ingredients['spicy'] == 'Y'])

        # each varies from roughly .5 to 1
        sweet_score = (n_sweet_lower/2 + n_sweet_upper)/5
        salty_score = (n_salty_lower/2 + n_salty_upper)*2/5
        sour_score = (n_sour_lower/2 + n_sour_upper)*2/5
        savory_score = (n_savory_lower/2 + n_savory_upper)*3/5
        bitter_score = (n_bitter_lower/2 + n_bitter_upper)*3/5
        spicy_score = (n_spicy_lower/2 + n_spicy_upper)*2/5

        flavor_balance_score = 5 / (1 + abs(1-sweet_score) + abs(1-salty_score) + abs(1-sour_score) + abs(1-savory_score) + abs(1-spicy_score)) - 1.25
    #     print(flavor_balance_score)

        score += flavor_balance_score
    #     print(sweet_score, salty_score, sour_score, savory_score, bitter_score, spicy_score)
    #     print()

    #     print(n_sweet_lower, n_sweet_upper)
    #     print(n_salty_lower, n_salty_upper)
    #     print(n_sour_lower, n_sour_upper)
    #     print(n_savory_lower, n_savory_upper)
    #     print(n_bitter_lower, n_bitter_upper)
    #     print(n_spicy_lower, n_spicy_upper)
    #     print()

    # TEXTURE BALANCE BONUS ============================================================================================
        # ranges from roughly (0 to 1) * .75
        n_crunchy_lower = len(selected_ingredients[selected_ingredients['salad_crunchy'] == 'y'])
        n_crunchy_upper = len(selected_ingredients[selected_ingredients['salad_crunchy'] == 'Y'])
        n_chewy_lower = len(selected_ingredients[selected_ingredients['salad_chewy'] == 'y'])
        n_chewy_upper = len(selected_ingredients[selected_ingredients['salad_chewy'] == 'Y'])
        n_juicy_lower = len(selected_ingredients[selected_ingredients['salad_juicy'] == 'y'])
        n_juicy_upper = len(selected_ingredients[selected_ingredients['salad_juicy'] == 'Y'])

        # each ranges from roughly 0 to 1
        crunchy_score = (n_crunchy_lower/2 + n_crunchy_upper)/3
        chewy_score = (n_chewy_lower/2 + n_chewy_upper)
        juicy_score = (n_juicy_lower/2 + n_juicy_upper)/3
    #     print(crunchy_score, chewy_score, juicy_score)

        texture_balance_score = 4 / (1 + abs(1-crunchy_score) + abs(1-chewy_score) + abs(1-juicy_score)) - 1
    #     print(texture_balance_score)
    #     print()

        score += texture_balance_score * .75

    # seems like it's hard to balance food groups on top of everything else. pity the scores aren't more independent
    # FOOD GROUP BALANCE BONUS =========================================================================================
        # ranges from roughly (25 to 1) * 2
        n_fruit = len(selected_ingredients[selected_ingredients['fruit'] == 'y'])
        n_veg = len(selected_ingredients[selected_ingredients['veg'] == 'y'])
        n_protein = len(selected_ingredients[selected_ingredients['protein'] == 'y'])

        # each varies from roughly 0 to 1 (sometimes a little over)
        fruit_score = n_fruit / 3
        veg_score = n_veg / 5
        protein_score = n_protein / 3
    #     print(fruit_score, veg_score, protein_score)

        food_group_balance_score = 3 / (1 + abs(1-fruit_score) + abs(1-veg_score) + abs(1-protein_score)) - .75
    #     print(food_group_balance_score)
    #     print()

        score += food_group_balance_score * 2

        if score > top_score:
            top_score = score
            top_food_group_balance_score = food_group_balance_score
            top_average_shortest_path_score = average_shortest_path_score
            top_flavor_balance_score = flavor_balance_score
            top_texture_balance_score = texture_balance_score
            top_all_clashing_pairs_score = all_clashing_pairs_score
            top_pair_strength_score = pair_strength_score
    #         top_upper_pairs = upper_pairs
    #         top_lower_pairs = lower_pairs
            top_lc_pairs = lower_category_pairs
            top_ld_pairs = lower_direct_pairs
            top_uc_pairs = upper_category_pairs
            top_ud_pairs = upper_direct_pairs
            top_selected_ingredients = selected_ingredients
            top_average_shortest_path_length = average_shortest_path_length
            top_all_clashing_pairs = all_clashing_pairs
    #         top_upper_proportion = len(upper_pairs) / (len(upper_pairs) + len(lower_pairs))
    # print('TOP AVG SHORTEST PATH LENGTH', top_average_shortest_path_length)
    # print('TOP UPPER PROPORTION', top_upper_proportion)
#     print('TOP AVERAGE SHORTEST PATH SCORE', top_average_shortest_path_score * 4)
#     print('TOP PAIR STRENGTH SCORE', top_pair_strength_score * 3)
#     print('TOP ALL CLASHING PAIRS SCORE', top_all_clashing_pairs_score * -1.5)
#     print('TOP FLAVOR BALANCE SCORE', top_flavor_balance_score)
#     print('TOP TEXTURE BALANCE SCORE', top_texture_balance_score * .75)
#     print('TOP FOOD GROUP BALANCE SCORE', top_food_group_balance_score * 2)
#     print('TOP_SCORE', top_score)  
    
    vegan = top_selected_ingredients['not_vegan'].sum() == ''
    gluten_free = top_selected_ingredients['gluten'].sum() == ''
    
    recipe_greens = top_selected_ingredients[top_selected_ingredients['salad_green'] == 'y']
    recipe_extras = top_selected_ingredients[top_selected_ingredients['salad_extra'] == 'y']
    recipe_dressing_oils = top_selected_ingredients[top_selected_ingredients['salad_dressing_oil'] == 'y']
    recipe_dressing_vinegars = top_selected_ingredients[top_selected_ingredients['salad_dressing_vinegar'] == 'y']
    recipe_dressing_salts = top_selected_ingredients[top_selected_ingredients['salad_dressing_salt'] == 'y']
    recipe_dressing_peppers = top_selected_ingredients[top_selected_ingredients['salad_dressing_pepper'] == 'y']
    recipe_dressing_garlics = top_selected_ingredients[top_selected_ingredients['salad_dressing_garlic'] == 'y']
    # could just select 'salad_dressing', but this includes garlics
    recipe_dressing = recipe_dressing_oils.append(recipe_dressing_vinegars).append(recipe_dressing_salts).append(recipe_dressing_peppers)

    new_recipe = pd.DataFrame({
        'vegan': [vegan],
        'gluten_free': [gluten_free],
        'basic': [salad_data_is_basic],
        'best_of': [best_of],
        'score': [top_score],
        'pairing_density_bonus': [top_average_shortest_path_score * 4],
        'pair_strength_bonus': [top_pair_strength_score * 3],
        'clash_penalty': [top_all_clashing_pairs_score * 4],
        'flavor_balance_bonus': [top_flavor_balance_score],
        'texture_balance_bonus': [top_texture_balance_score * 75],
        'food_group_balance_bonus': [top_food_group_balance_score * 2],
        'lc_pairs': [top_lc_pairs],
        'ld_pairs': [top_ld_pairs],
        'uc_pairs': [top_uc_pairs],
        'ud_pairs':[top_ud_pairs],
        'clashing_pairs': [top_all_clashing_pairs],
        'ingredient_names': [top_selected_ingredients['name'].values.tolist()],
        'leafy_green_names': [recipe_greens['name'].values.tolist()],
        'extra_names': [recipe_extras['name'].values.tolist()],
        'dressing_names': [recipe_dressing['name'].values.tolist()],
    })
    salad_recipe_data = salad_recipe_data.append(new_recipe, sort=False)
    print('SALAD RECIPE RECORDED. SCORE:', top_score)

SALAD RECIPE RECORDED. SCORE: 10.84495599436616
SALAD RECIPE RECORDED. SCORE: 10.380796536796538
SALAD RECIPE RECORDED. SCORE: 9.970445344129555
SALAD RECIPE RECORDED. SCORE: 10.502565236620963
SALAD RECIPE RECORDED. SCORE: 9.951428571428572
SALAD RECIPE RECORDED. SCORE: 10.649220032840724
SALAD RECIPE RECORDED. SCORE: 10.501343215358496
SALAD RECIPE RECORDED. SCORE: 10.928643351805565
SALAD RECIPE RECORDED. SCORE: 10.406142057710003
SALAD RECIPE RECORDED. SCORE: 10.057016317016318
SALAD RECIPE RECORDED. SCORE: 10.507801418439714
SALAD RECIPE RECORDED. SCORE: 11.17917030740409
SALAD RECIPE RECORDED. SCORE: 11.152702736467386
SALAD RECIPE RECORDED. SCORE: 11.853302611367129
SALAD RECIPE RECORDED. SCORE: 10.670695970695972
SALAD RECIPE RECORDED. SCORE: 10.918930307165601
SALAD RECIPE RECORDED. SCORE: 11.416666666666666
SALAD RECIPE RECORDED. SCORE: 11.064540582224794
SALAD RECIPE RECORDED. SCORE: 10.0177140101278
SALAD RECIPE RECORDED. SCORE: 10.338076923076922
SALAD RECIPE RECORDED. SCO

In [22]:
from datetime import date
today = date.today()
date_string = f'{str(today.year)}_{str(today.month)}_{str(today.day)}'

salad_recipe_data.to_csv(os.path.join(root_path, 'DATA/salad_recipe_data_latest.csv'), index=False)
salad_recipe_data.to_csv(os.path.join(root_path, f'DATA/salad_recipe_data_{date_string}.csv'), index=False)

### 4.1.3. Displaying records

In [28]:
# !pip install pyvis

from pyvis import network as net

In [35]:
recipe = salad_recipe_data[(salad_recipe_data['basic'] == True) & (salad_recipe_data['vegan'] == True) & salad_recipe_data['gluten_free'] == True].sample(1).iloc[0]
recipe_net = net.Network(notebook=True)

nodes = recipe['ingredient_names']

nodes_color = []
for name in recipe['ingredient_names']:
    ingredient = salad_data[salad_data['name'] == name].iloc[0]
    if ingredient['salad_green'] == 'y':
        nodes_color.append('lightgreen')
    elif ingredient['salad_extra'] == 'y':
        if ingredient['veg'] == 'y':
            nodes_color.append('green')
        elif ingredient['fruit'] == 'y':
            nodes_color.append('orange')
        elif ingredient['protein_nut_seed'] == 'y':
            nodes_color.append('brown')
        else:
            nodes_color.append('lightblue')
    elif ingredient['salad_dressing'] == 'y':
        nodes_color.append('lightgrey')  
        
recipe_net.add_nodes(
    nodes=nodes,
    color=nodes_color
)

for pair in recipe['lc_pairs']:
    recipe_net.add_edge(pair[0], pair[1], physics=False, color='lightgrey')

for pair in recipe['ld_pairs']:
    recipe_net.add_edge(pair[0], pair[1], physics=False, color='grey')
    
for pair in recipe['uc_pairs']:
    recipe_net.add_edge(pair[0], pair[1], color='darkgrey')
    
for pair in recipe['ud_pairs']:
    recipe_net.add_edge(pair[0], pair[1], color='black')

if not recipe['vegan']:
    print('NOT VEGAN')
if not recipe['gluten_free']:
    print('CONTAINS GLUTEN')
print('SCORE:', recipe['score'])
recipe_net.show('recipe_net.html')

SCORE: 10.28669499836548


## 4.2. Random control

In [16]:
# ALGORITHM B (random control)
n_greens = random.randrange(1, 4)
n_extras = random.randrange(0, 10)
n_dressing_oils = 1
n_dressing_vinegars = 1
n_dressing_salts = 1
n_dressing_peppers = 1
# n_dressing_garlics = random.randrange(0, 2) # maybe make presence dependent on the rest. or, just leave out for now.

selected_greens = salad_greens.sample(n_greens)
selected_extras = salad_extras.sample(n_extras)
selected_dressing_oils = salad_dressing_oils.sample(n_dressing_oils)
selected_dressing_vinegars = salad_dressing_vinegars.sample(n_dressing_vinegars)
selected_dressing_salts = salad_dressing_salts.sample(n_dressing_salts)
selected_dressing_peppers = salad_dressing_peppers.sample(n_dressing_peppers)

selected_ingredients = selected_greens.append(selected_extras).append(selected_dressing_oils).append(selected_dressing_vinegars).append(selected_dressing_salts).append(selected_dressing_peppers)

upper_pairs = []
lower_pairs = []
for ingredient_name in selected_ingredients['name']:
    for pairs_with_name in pairing_data['upper_pairs_with_names'][pairing_data['name'] == ingredient_name].iloc[0]:
        if pairs_with_name in selected_ingredients['name'].values.tolist():
#             print('UPPER PAIR:', ingredient_name, 'WITH', pairs_with_name)
            upper_pairs.append([ingredient_name, pairs_with_name])
    for pairs_with_name in pairing_data['lower_pairs_with_names'][pairing_data['name'] == ingredient_name].iloc[0]:
        if pairs_with_name in selected_ingredients['name'].values.tolist():
#             print('LOWER PAIR:', ingredient_name, 'WITH', pairs_with_name)
            lower_pairs.append([ingredient_name, pairs_with_name])

score = (len(upper_pairs) * 3 + len(lower_pairs)) / (n_greens + n_extras)
print('RANDOM SCORE', score)



random_net = net.Network(notebook=True)

nodes = selected_ingredients['name'].tolist()

def get_color(row):
#     print(type(row))
#     print(row)
    if row['salad_green'] == 'y':
        return 'lightgreen'
    elif row['salad_extra'] == 'y':
        if row['veg'] == 'y':
            return 'green'
        elif row['fruit'] == 'y':
            return 'orange'
        elif row['protein_nut_seed'] == 'y':
            return 'brown'
        else:
            return 'lightblue'
    elif row['salad_dressing'] == 'y':
        return 'lightgrey'
    
nodes_color = selected_ingredients.apply(get_color, axis=1).tolist()

random_net.add_nodes(
    nodes=nodes,
    color=nodes_color
)


for pair in upper_pairs:
    random_net.add_edge(pair[0], pair[1], color='black')

for pair in lower_pairs:
    random_net.add_edge(pair[0], pair[1], physics=False, color='lightgrey')

random_net.show('selected_net.html')

RANDOM SCORE 11.875
